In [24]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

In [25]:
seed = 0
# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [26]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo2.csv")
dog_train=pd.read_csv("trainDoginfo2.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

dog_data
# print(dog_train)
# print(kindCd)

,neuterYn,sexCd,weight,noticeDays,age2,processState
0,N,F,7.46,10,12,0
1,N,M,7.00,14,1,1
2,U,M,4.50,11,2,0
3,N,M,10.00,8,1,0
4,N,M,6.00,8,4,0
...,...,...,...,...,...,...
22787,N,M,1.00,10,0,1
22788,N,M,1.00,10,0,1
22789,N,M,1.00,10,0,1
22790,U,F,6.00,12,3,0


In [28]:
# kindCd = np.array(kindCd_data, dtype = np.float64)
# kindCd_train=np.array(kindCd_data, dtype=np.float64)

# kindCd = kindCd.reshape(177)
# kindCd_train=kindCd_train.reshape(177)
# print(kindCd)
# print(kindCd.shape)

In [29]:
# # kindNum을 원핫 인코딩
# kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# # kindCd

# # 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_data.drop(['kindNum'], axis='columns', inplace=True)
# dog_data = pd.concat([dog_data, kindCd], axis=1)


# # kindNum을 원핫 인코딩
# kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# # 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_train.drop(['kindNum'], axis='columns', inplace=True)
# dog_train = pd.concat([dog_train, kindCd_train], axis=1)
# dog_data

In [30]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [31]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)
data.head()
train_data.head()

print(data)

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)


       weight  noticeDays  age2  processState  neuterYn_N  neuterYn_U  \
0        7.46          10    12             0           1           0   
1        7.00          14     1             1           1           0   
2        4.50          11     2             0           0           1   
3       10.00           8     1             0           1           0   
4        6.00           8     4             0           1           0   
...       ...         ...   ...           ...         ...         ...   
22787    1.00          10     0             1           1           0   
22788    1.00          10     0             1           1           0   
22789    1.00          10     0             1           1           0   
22790    6.00          12     3             0           0           1   
22791    3.50          10     0             0           1           0   

       neuterYn_Y  sexCd_F  sexCd_M  sexCd_Q  
0               0        1        0        0  
1               0        0   

In [32]:
a = data[:, :3]
b = data[:, 4:]

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

print(xData)
type(xData)
xData.shape

[[ 7.46 10.   12.   ...  1.    0.    0.  ]
 [ 7.   14.    1.   ...  0.    1.    0.  ]
 [ 4.5  11.    2.   ...  0.    1.    0.  ]
 ...
 [ 1.   10.    0.   ...  0.    1.    0.  ]
 [ 6.   12.    3.   ...  1.    0.    0.  ]
 [ 3.5  10.    0.   ...  0.    1.    0.  ]]


(22792, 9)

In [33]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]

print(yData)
type(yData)
yData.dtype

[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


dtype('float64')

In [34]:
#뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만듭니다.
X=tf.placeholder(tf.float64, [None, 9])
Y=tf.placeholder(tf.float64, [None, 1])

#다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화합니다.
a=tf.Variable(tf.random_normal([9, 1], dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
b=tf.Variable(tf.random_normal([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.
print(X)
print(Y)


Tensor("Placeholder_4:0", shape=(?, 9), dtype=float64)
Tensor("Placeholder_5:0", shape=(?, 1), dtype=float64)


In [38]:
# 시그모이드 방정식을 만듭니다.
y = tf.sigmoid(tf.matmul(X, a) + b)
# 오차 함수
cost = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))
optimizer=tf.train.GradientDescentOptimizer(0.05).minimize(cost)

predicted = tf.cast(y >= 0.5, dtype = tf.float64)

# 예상값이 실제값과 일치하는 정도(정확도)를 계산합니다.
# tf.equal(): tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False 결과를 tf.float64로 캐스팅 시키고 시행된 전체 평균을 계산합니다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float64))  # 정확도 (0 또는 1) 

with tf.Session() as sess:  # 세션 구간을 만듦
    sess.run(tf.global_variables_initializer())
    for i in range(10001):
        # count : 학습 횟수, a1 : 기울기(공부 시간), asess.run(cost, feed_dict={X:xData, Y:yData}))
        a_, b_, cost_, _ = sess.run([a, b, cost, optimizer], feed_dict = {X:xData, Y:yData})
        if i % 500 == 0:
            print(sess.run(y, feed_dict={X:xData}))
            print('count:{}, b = {}, loss = {}'.format(i, b_, cost_)) # 학습시킨 데이터 출력
    av=sess.run([accuracy], feed_dict={X:xData, Y:yData})
    print("정확도: {}".format(av))
                
    for i in range(len(train_data)):
        new_x=train_data[i, :].reshape(1,9)
        result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
        print('입양 여부 : %s,    입양 확률 : %8.4f%%' % ('입양' if result == 1 else '안락사', new_y*100))
    
    saver = tf.train.Saver()
    save_path = saver.save(sess, './dog.ckpt')
    print('학습된 모델을 저장했습니다')

[[3.28942444e-02]
 [4.63038172e-09]
 [2.66655686e-07]
 ...
 [3.26815684e-05]
 [2.88253622e-07]
 [4.43611680e-06]]
count:0, b = [1.26315695], loss = 7.6169004513270835
[[0.51807752]
 [0.30835708]
 [0.36534961]
 ...
 [0.55486752]
 [0.33257849]
 [0.53010126]]
count:500, b = [1.14810645], loss = 0.6850109930553505
[[0.44836973]
 [0.33498535]
 [0.43321819]
 ...
 [0.54894477]
 [0.36516329]
 [0.52443423]]
count:1000, b = [0.97977858], loss = 0.6790283844922566
[[0.4410555 ]
 [0.35390255]
 [0.4404051 ]
 ...
 [0.54391452]
 [0.37586058]
 [0.52010264]]
count:1500, b = [0.84534113], loss = 0.6774473389930782
[[0.54061154]
 [0.47411268]
 [0.52426204]
 ...
 [0.60241081]
 [0.4748406 ]
 [0.58840804]]
count:2000, b = [0.74414095], loss = 0.6956124434501187
[[0.54316399]
 [0.48893595]
 [0.52688576]
 ...
 [0.60159609]
 [0.48243317]
 [0.58814044]]
count:2500, b = [0.66160236], loss = 0.6964085320623876
[[0.5451347 ]
 [0.50036843]
 [0.52893143]
 ...
 [0.60086437]
 [0.4883324 ]
 [0.58781044]]
count:3000, b 

입양 여부 : 안락사,    입양 확률 :  49.8819%
입양 여부 : 입양,    입양 확률 :  58.1244%
입양 여부 : 입양,    입양 확률 :  56.9114%
입양 여부 : 입양,    입양 확률 :  52.2368%
입양 여부 : 입양,    입양 확률 :  56.1189%
입양 여부 : 입양,    입양 확률 :  58.2449%
입양 여부 : 입양,    입양 확률 :  58.2449%
입양 여부 : 입양,    입양 확률 :  57.5090%
입양 여부 : 입양,    입양 확률 :  55.0377%
입양 여부 : 입양,    입양 확률 :  59.4658%
입양 여부 : 입양,    입양 확률 :  58.9762%
입양 여부 : 입양,    입양 확률 :  53.7686%
입양 여부 : 입양,    입양 확률 :  56.3588%
입양 여부 : 입양,    입양 확률 :  56.7421%
입양 여부 : 입양,    입양 확률 :  54.9895%
입양 여부 : 입양,    입양 확률 :  54.9895%
입양 여부 : 입양,    입양 확률 :  56.6549%
입양 여부 : 안락사,    입양 확률 :  49.5323%
입양 여부 : 입양,    입양 확률 :  50.7641%
입양 여부 : 입양,    입양 확률 :  54.1120%
입양 여부 : 입양,    입양 확률 :  53.7145%
입양 여부 : 입양,    입양 확률 :  54.0093%
입양 여부 : 입양,    입양 확률 :  60.0554%
입양 여부 : 입양,    입양 확률 :  51.8722%
입양 여부 : 입양,    입양 확률 :  53.6099%
입양 여부 : 안락사,    입양 확률 :  41.1218%
입양 여부 : 안락사,    입양 확률 :  38.5941%
입양 여부 : 입양,    입양 확률 :  54.4990%
입양 여부 : 입양,    입양 확률 :  53.9918%
입양 여부 : 입양,    입양 확률 :  56.1466%
입양 여부 

학습된 모델을 저장했습니다
